In [12]:
import jax
import genjax
import pickle

key = jax.random.PRNGKey(314159)

@genjax.gen
def model():
    x = genjax.normal(0.0, 1.0) @ "x"
    return x

In [13]:
import dill as pickle

def to_tuple(obj):
    return (obj.__class__, obj.flatten())


def pickle_serialize(obj):
    """Serializes an object using pickle."""
    return pickle.dumps(to_tuple(obj))


def pickle_deserialize(serialized_obj):
    """Deserializes an object using pickle."""
    cls, (xs, data) = pickle.loads(serialized_obj)
    return cls.unflatten(xs, data)

In [27]:
key, sub_keys = genjax.slash(key, 10000)
_, tr = jax.vmap(model.simulate, in_axes=(0, None))(sub_keys, ())
tr["x"]

Array([-1.8959491 ,  0.76408625,  1.4928106 , ..., -1.2883965 ,
        0.32632154,  1.0351807 ], dtype=float32)

In [28]:
s = pickle_serialize(tr)

In [29]:
dtr = pickle_deserialize(s)

In [30]:
dtr

BuiltinTrace(gen_fn=BuiltinGenerativeFunction(source=PytreeClosure(callable=<function model at 0x10397b240>, environment=[])), args=(), retval=Array([-1.8959491 ,  0.76408625,  1.4928106 , ..., -1.2883965 ,
        0.32632154,  1.0351807 ], dtype=float32), choices=Trie(inner={'x': DistributionTrace(gen_fn=Normal(), args=(Array([0., 0., 0., ..., 0., 0., 0.], dtype=float32, weak_type=True), Array([1., 1., 1., ..., 1., 1., 1.], dtype=float32, weak_type=True)), value=Array([-1.8959491 ,  0.76408625,  1.4928106 , ..., -1.2883965 ,
        0.32632154,  1.0351807 ], dtype=float32), score=Array([-2.71625  , -1.2108524, -2.0331802, ..., -1.7489213, -0.9721814,
       -1.454738 ], dtype=float32))}), cache=Trie(inner={}), score=Array([-2.71625  , -1.2108524, -2.0331802, ..., -1.7489213, -0.9721814,
       -1.454738 ], dtype=float32))

In [31]:
dtr == tr

False

In [33]:
all(dtr.get_choices()["x"] == tr["x"])

True

In [34]:
tr.get_gen_fn()

BuiltinGenerativeFunction(source=PytreeClosure(callable=<function model at 0x12ab9ae80>, environment=[]))

In [ ]:
deserialize(gen_fn:: T, data_ptr, backend :: B)

In [ ]:
backend.serialize(tr :: Pytree)
backend.deserialize(gen_fn, data_ptr)